In [147]:
import urbs

import pandas as pd
import numpy as np

In [148]:
year = 2020

In [149]:
# Read input Excel to get the file format
xls = pd.ExcelFile(f'myopic/input/input_bayern_{year}.xlsx')

input_file = {}
for sheet in xls.sheet_names:
    input_file[sheet] = xls.parse(sheet_name = sheet, index_col = 0)

In [150]:
# Import timeseries data

xls = pd.ExcelFile(f'myopic/input/timeseries.xlsx')

timeseries = {}
for sheet in xls.sheet_names:
    timeseries[sheet] = xls.parse(sheet_name = sheet, index_col = 0)

In [151]:
#output_file = urbs.load(f'myopic/output/output_bayern_{year}.h5')._result
#urbs_cap_new = output_file['cap_pro_new'].unstack()

In [152]:
# Update 'Global' sheet
input_file['Global'].at['Support timeframe', 'value'] = year + 5
input_file['Global'].at['CO2 limit', 'value'] = timeseries['co2_budget'].at['CO2 budget', year + 5]

In [153]:
# Update 'Commodity'

input_file['Commodity'] = input_file['Commodity'].reset_index()
timeseries['prices'] = timeseries['prices'].reset_index()

input_file['Commodity']['price'] = timeseries['prices'][year+5]
input_file['Commodity'] = input_file['Commodity'].set_index('Site')
input_file['Commodity'].head()

,Commodity,Type,price,max,maxperhour
Site,,,,,
Bayern,Water,SupIm,NaN,NaN,NaN
Bayern,Wind,SupIm,NaN,NaN,NaN
Bayern,Solar,SupIm,NaN,NaN,NaN
Bayern,CO2,Env,0.0,inf,inf
Bayern,Geothermie,Stock,0.0,inf,inf


In [154]:
# Update 'Process' sheet

input_file['Process'] = input_file['Process'].reset_index()
timeseries['retiring'] = timeseries['retiring'].reset_index()

input_file['Process']['inst-cap'] = timeseries['retiring'][year+5]

input_file['Process'] = input_file['Process'].set_index('Site')
input_file['Process'].head()

,Process,inst-cap,lifetime,cap-lo,cap-up,max-grad,min-fraction,inv-cost,fix-cost,var-cost,startup-cost,wacc,depreciation,area-per-cap
Site,,,,,,,,,,,,,,
Bayern,Hydro plant,1981.00,10,0,1981.0,1,0,2208000,22080.000,0.00,0,0.07,60,NaN
Bayern,Onshore wind park,1899.75,25,0,inf,1,0,1118770,13984.625,1.50,0,0.07,25,NaN
Bayern,Photovoltaics,10526.25,25,0,inf,1,0,733470,11588.826,0.01,0,0.07,25,NaN
Bayern,Biomass plant,1848.00,30,0,1848.0,1,0,2209000,103823.000,0.00,0,0.07,30,NaN
Bayern,Geothermal plant,46.00,40,0,46.0,1,0,3392000,80051.200,0.00,0,0.07,40,NaN


In [155]:
# Update 'Storage' sheet

#input_file['Storage']

In [156]:
# Update 'Demand' sheet
# only woodcutter approach, will loop through demand type commodities, standarized names in 'timeseries', and catch-try

input_file['Demand']['Bayern.Space heating'] = timeseries['profiles']['profile_space_heating'] * timeseries['demand'].at['Space heating', year+5] * 1e6
input_file['Demand']['Bayern.Process heating A'] = timeseries['profiles']['profile_process_heating'] * timeseries['demand'].at['Process heating A', year+5] * 1e6
input_file['Demand']['Bayern.Process heating B'] = timeseries['profiles']['profile_process_heating'] * timeseries['demand'].at['Process heating B', year+5] * 1e6
input_file['Demand']['Bayern.Process heating C'] = timeseries['profiles']['profile_process_heating'] * timeseries['demand'].at['Process heating C', year+5] * 1e6
input_file['Demand']['Bayern.Electric charging'] = timeseries['profiles']['profile_charging'] * timeseries['demand'].at['Electric charging', year+5] * 1e6
input_file['Demand']['Bayern.Elec_Rest'] = timeseries['profiles']['profile_electricity'] * timeseries['demand'].at['Electricity', year+5] * 1e6

In [157]:
# Write DataFrame dictionary into a 
with pd.ExcelWriter(f"myopic/input/input_bayern_{year+5}.xlsx") as writer:
    for sheet in list(input_file.keys()):
        input_file[sheet].to_excel(writer, sheet_name = sheet)